In [1]:
'''
Author: Satya Katragadda
'''

In [ ]:
import pandas as pd
import seaborn as sea
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.stats import pearsonr
from sklearn import linear_model as lm

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/State_Consolidated_Mobility_Miles_New_Actionable_Oct20.csv')
df.fillna(0, inplace=True)



In [4]:
def generateCorrelations(df):
    try:
        p = pearsonr(df['desCartes'].values, df['Incidence'].values)[0]
        q = pearsonr(df['desCartes'].values, df['Incidence'].values)[1]
        return p,q
    except:
        return -1,-1


def enterDict(state, lag, phase, correlation, sig):
    row = {}
    row['state'] = state
    row['lag'] = lag
    row['Phase'] = phase
    row['correlation'] = correlation
    row['sig'] = sig
    return row


def normalize(series):
    return (series-series.min())/(series.max()-series.min())

In [5]:
states = df['state'].unique()
phdf = pd.read_csv('Data/phases_phase1.csv')
ndf = {}

sdf = pd.DataFrame()
k = 0
lags = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]
for state in states:
    print(state)
    for lag in lags:

        tdf = df[df['state']==state]
        tdf['desCartes'] = tdf['desCartes'].shift((lag*-1))
        tdf = tdf[~tdf['desCartes'].isin([np.nan, np.inf, -np.inf])]
        tdf = tdf[~tdf['Incidence'].isin([np.nan, np.inf, -np.inf])]
        row={}
        row['state'] = state
        row['Phase'] = 'CompleteSeries'
        row['correlation'] = pearsonr(tdf['desCartes'].values, tdf['Incidence'].values )[0]
        row['sig'] = pearsonr(tdf['desCartes'].values, tdf['Incidence'].values )[1]
        row['lag'] = lag
        ndf[k] =row
        k = k+1
        tdf.sort_values('startDayWeek', inplace=True)
        if(state  in phdf.columns):
            tphdf = phdf[['Phase Loc',state]]
            p1_start = tphdf[tphdf['Phase Loc']=='Phase1'][state].values[0]
            p1_end = tphdf[tphdf['Phase Loc']=='Phase2'][state].values[0]
            p2_start = tphdf[tphdf['Phase Loc']=='Phase2'][state].values[0]
            p2_end = tphdf[tphdf['Phase Loc']=='Phase3'][state].values[0]
            p3_start = tphdf[tphdf['Phase Loc']=='Phase3'][state].values[0]
            p3_end = tphdf[tphdf['Phase Loc']=='Phase4'][state].values[0]
            p4_start = tphdf[tphdf['Phase Loc']=='Phase4'][state].values[0]
            p4_end = tphdf[tphdf['Phase Loc']=='Phase5'][state].values[0]
            p5_start = tphdf[tphdf['Phase Loc']=='Phase5'][state].values[0]
            p5_end = tphdf[tphdf['Phase Loc']=='End'][state].values[0]
            if p1_end!=0:
                ptdf = tdf[(tdf['week']>=p1_start) & (tdf['week']<=p1_end)]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase1', p, q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p1_start]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase1', p, q)
                k = k + 1
                continue
            if p2_end!=0:
                ptdf = tdf[(tdf['week']>=p2_start) & (tdf['week']<=p2_end)]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase2', p, q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p2_start]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase2', p, q)
                k = k + 1
                continue
            if p3_end!=0:
                ptdf = tdf[(tdf['week']>=p3_start) & (tdf['week']<=p3_end)]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase3', p, q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p3_start]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase3', p, q)
                k = k + 1
                continue
            if p4_end!=0:
                ptdf = tdf[(tdf['week']>=p4_start) & (tdf['week']<=p4_end)]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase4', p, q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p4_start]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase4', p, q)
                k = k + 1
                continue
            if p5_end!=0:
                ptdf = tdf[(tdf['week']>=p5_start) & (tdf['week']<=p5_end)]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase5', p, q)
                k = k + 1
            else:
                ptdf = tdf[tdf['week']>=p5_start]
                p ,q= generateCorrelations(ptdf)
                ndf[k] = enterDict(state, lag, 'Phase5', p, q)
                k = k + 1
                continue

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


In [6]:
cdf = pd.DataFrame.from_dict(ndf, 'index')

cdf_F = cdf.pivot_table(index='Phase', columns=['state','lag'], values='correlation')
cdf_F.to_csv('Phases_Casality_All_States_Corr_12302020.csv')

cdf_P = cdf.pivot_table(index='Phase', columns=['state','lag'], values='Sig')
cdf_P.to_csv('Phases_Casality_All_States_PValue_12302020.csv')

In [9]:
writer = pd.ExcelWriter('Correlation_DesCartes_Incidence_Phases_Correlation_01112020.xlsx', engine='xlsxwriter')
ncdf = pd.DataFrame.from_dict(ndf, 'index')
phases = ncdf['Phase'].unique()
stateFrame = pd.read_csv('Data/PeakMonths.csv')
for phase in phases:
    twrt = ncdf[ncdf['Phase']==phase]
    twrt_pivot = twrt.pivot_table(index='state', columns='lag', values='correlation').reset_index()
    #print(state, phase)
    #print(twrt_pivot.head())
    #print(stateFrame.head())
    twrt_pivot = pd.merge(stateFrame, twrt_pivot, how='left')
    twrt_pivot.sort_values('state', inplace=True)
    twrt_pivot.to_excel(writer, index=False, sheet_name=phase)

writer.save()

In [8]:

writer = pd.ExcelWriter('Correlation_DesCartes_Incidence_Phases_PValue_01112020.xlsx', engine='xlsxwriter')
ncdf = pd.DataFrame.from_dict(ndf, 'index')
phases = ncdf['Phase'].unique()
stateFrame = pd.read_csv('Data/PeakMonths.csv')
for phase in phases:
    twrt = ncdf[ncdf['Phase']==phase]
    twrt_pivot = twrt.pivot_table(index='state', columns='lag', values='sig').reset_index()
    #print(state, phase)
    #print(twrt_pivot.head())
    #print(stateFrame.head())
    twrt_pivot = pd.merge(stateFrame, twrt_pivot, how='left')
    twrt_pivot.sort_values('state', inplace=True)
    twrt_pivot.to_excel(writer, index=False, sheet_name=phase)

writer.save()